# Bing Local Business API

Be sure to check out them official documentation [here.](https://docs.microsoft.com/en-us/azure/cognitive-services/bing-local-business-search/local-search-reference)   <br>

**EXTREMELY IMPORTANT NOTE:**
There is no "Bing local business" API in the Azure portal. You need a **Bing Search v7 API** subscription from https://portal.azure.com to use the local business endpoint, and you need to select the **S10 pricing tier**.  

If you don't, nothing will work and you'll probably get really frustrated.

# Index
- [Setup](#Setup)
- [Search Business Listings with a Query](#Search-Business-Listings-with-a-Query)
- [Paging](#Paging)
- [Search Business Listings Categorically](#Search-Business-Listings-Categorically)

## Setup

#### _Imports & Constants_:

In [1]:
import requests
import os
from pprint import pprint
from localbusiness_functions import *

In [2]:
API_KEY = os.environ.get('BING_KEY', default='ENTER YOUR API KEY HERE IF YOU DIDNT SET BING_KEY')

In [3]:
API_KEY = '637700676f0742e6a688ceeead973fe9'

In [4]:
BASE = 'https://api.cognitive.microsoft.com/bing/v7.0/'
SUFFIX = 'localbusinesses/search?'
ENDPOINT = BASE + SUFFIX

In [5]:
STARTING_HEADERS = {
    'Ocp-Apim-Subscription-Key' : API_KEY
}
STARTING_PARAMS = {
    "q" : None,
    "mkt" : "en-US",
    "offset" : "0",
    "count" : "50"
  }

HEADERS = STARTING_HEADERS.copy()
PARAMS = STARTING_PARAMS.copy()

#### _Functions_:

In [6]:
# Common accross all notebooks - 
def call_api(query, session=None, custom_params=None, check_session=False):
    sesh = session
    if not session:
        sesh = requests.Session()
        sesh.params = STARTING_PARAMS.copy()
        sesh.params.update({'q' : query})
        sesh.headers = STARTING_HEADERS.copy()
    if custom_params:
        sesh.params.update(custom_params)
    if check_session:
        return (sesh.params, sesh.headers)
    return sesh.get(ENDPOINT)

def handle_response(resp):
    assert resp.status_code == 200
    return resp.json()

In [7]:
# restating a few parsers. These can also be found in .localbusiness_functions.py
def get_listings(json):
    return [i for i in json['places']['value']]

def get_names(json):
    return [i['name'] for i in json['places']['value']]

def get_cities(json):
    return [i['address']['addressLocality'] for i in json['places']['value']]

In [8]:
# misc
def get_tem(json):
    return json['places']['totalEstimatedMatches']

def dedupe(some_list):
    return list(set(some_list))

In [9]:
# output adapters
def full_output(json):
    return get_listings(json)

def summarized_output(json):
    return list(zip(get_names(json), get_cities(json)))

In [10]:
# helpers
def get_json_from_bing(q, session=None, custom_params=None):
    resp = call_api(q, session=session, custom_params=custom_params)
    json = handle_response(resp)
    return json

def get_business_listings(q, session=None, custom_params=None, summarized=True):
    json = get_json_from_bing(q=q, session=session, custom_params=custom_params)
    if not summarized:
        return full_output(json)
    return summarized_output(json)

def get_total_estimated_matches(q, session=None, custom_params=None):
    json = get_json_from_bing(q, session=session, custom_params=custom_params)
    return get_tem(json)
    
def check_session(q, session=None, custom_params=None):
    return call_api(q, session=session, custom_params=custom_params, check_session=True)

# Search Business Listings with a Query

This way of using the local business endpoint mirrors the typical search API experience. 

Let's start with something simple:

In [11]:
q = 'grocery store Seattle'

And that's pretty much all we need. let's quickly check our headers and params.

In [12]:
params, headers = check_session(q)
pprint(params)
print('\n')
pprint(headers)

{'count': '50', 'mkt': 'en-US', 'offset': '0', 'q': 'grocery store Seattle'}


{'Ocp-Apim-Subscription-Key': '637700676f0742e6a688ceeead973fe9'}


Double check that you see the correct `q` value within your `params` dictionary above, and that your `headers` contains a valid API key.

In [13]:
listings = get_business_listings(q)
pprint(listings)
print('{} businesses returned with this query'.format(len(listings)))

[('Kress IGA Supermarket', 'Seattle'),
 ('Uwajimaya', 'Seattle'),
 ('Pike Grocery', 'Seattle'),
 ('Paris Grocery', 'Seattle'),
 ('Yesler Grocery', 'Seattle'),
 ("Dan's Belltown Grocery", 'Seattle'),
 ("Lyon's Grocery", 'Seattle'),
 ('Stockbox Neighborhood Grocery', 'Seattle'),
 ('Grocery Outlet Bargain Market', 'Seattle'),
 ('Market Grocery & Deli', 'Seattle'),
 ("Wally's Grocery", 'Seattle'),
 ('Semiras Grocery', 'Seattle'),
 ("Noah's Grocery", 'Seattle'),
 ('Rotary Grocery', 'Seattle'),
 ('Grocery Outlet Bargain Market', 'Seattle'),
 ('Grocery Outlet Bargain Market', 'Seattle'),
 ("Han's Deli & Grocery", 'Seattle'),
 ('Mexican Grocery', 'Seattle'),
 ('Grocery Outlet Bargain Market', 'Seattle'),
 ('Cowen Park Grocery', 'Seattle'),
 ('Grocery Outlet Bargain Market', 'Seattle'),
 ('Vientian Asian Grocery', 'Seattle'),
 ('New Seasons Market', 'Seattle'),
 ('Grocery Outlet Bargain Market', 'Seattle'),
 ('Pacific Herb & Grocery', 'Seattle'),
 ('Union Park Grocery & Deli', 'Seattle'),
 ('Ci

The output you're seeing is summarized to give you a high-level view of the entities being returned. Let's take a look at what a full listing looks like:

In [45]:
full_listings = get_business_listings(q, summarized=False)
pprint(full_listings[0])

{'_type': 'LocalBusiness',
 'address': {'addressCountry': 'US',
             'addressLocality': 'Seattle',
             'addressRegion': 'WA',
             'neighborhood': 'Downtown',
             'postalCode': '98104',
             'streetAddress': '600 5th Ave S',
             'text': '600 5th Ave S, Seattle, WA, 98104'},
 'entityPresentationInfo': {'entityScenario': 'ListItem',
                            'entityTypeHints': ['Place', 'LocalBusiness']},
 'geo': {'latitude': 47.596839904785156, 'longitude': -122.3270034790039},
 'id': 'https://api.cognitive.microsoft.com/api/v7/#Places.0',
 'name': 'Uwajimaya',
 'routablePoint': {'latitude': 47.596839904785156,
                   'longitude': -122.3270034790039},
 'telephone': '(206) 624-6248',
 'url': 'https://www.uwajimaya.com/stores/seattle'}


# Paging

The local business API is an offshot of a major public search engine and it's still in preview, so pagination might be a bit tricky. Let's take a look at our returned `totalEstimatedMatches` (tem) value:

In [ ]:
total_estimated_matches = get_total_estimated_matches(q)

print('Bing says there are {} listings that match our query'.format(total_estimated_matches))
print('Our list currently has {} listings.'.format(len(listings)))

There's no telling which of the numbers above will be bigger. Bing might say there are less possible matches than it gave you, it might say there are more than it was willing to return, or it might tell you that you've been handed exactly as many listings as exist.

When the `totalEstimatedMatches` figure is <100, it is generally not to be trusted.  It's worth paging to the next set of results to see if we can extend our list.

let's increment our `offset` url param & try again:

In [ ]:
before = len(listings)

In [ ]:
new_params = {'offset' : str(len(listings)), 'count': "50"}
listings += get_business_listings(q, custom_params=new_params)
listings = dedupe(listings)#<==This is a simple way to make sure we don't accept duplicates, however it does not preserve ordering.

In [ ]:
after = len(listings)

Let's check out our results:

In [ ]:
pprint(listings)
print()
print('Previously had {} listings, added {} new ones for {} current listings. Bing says {} listings exist'.format(before, after - before, after, total_estimated_matches))

How you define your stop condition for pagination is up to you. This section is merely meant to put emphasis on the word **estimate** in `totalEstimatedMatches`.

# Search Business Listings Categorically

Within this folder you'll find a python file called [local_business_categories.py](./local_business_categories.py), which contains a list of possible values for the 'localCategories' url param (as of June 2019.) Refer to [the official docs](https://docs.microsoft.com/en-us/azure/cognitive-services/bing-local-business-search/local-categories) for an up to date list. 

In [14]:
from local_business_categories import MAIN_CATEGORIES

In [15]:
pprint(MAIN_CATEGORIES)

['EatDrink',
 'SeeDo',
 'Shop',
 'HotelsAndMotels',
 'BanksAndCreditUnions',
 'Parking',
 'Hospitals']


We can use one of these instead of our normal `q` param.

In [16]:
# null out q and set category/location
q = ''
newer_params = {'localCategories' : MAIN_CATEGORIES[-1]}# AKA {'localCategories' : 'Hospitals'}

In fact, `q` needs to be an empty string when searching categorically.

Just to be overly explicit & anal, note we had two pieces of information in our `q` value from the previous section: "grocery store Seattle"
 - A type of business ("Grocery Store")
 - A location ("Seattle")

Now we're looking for hospitals, and the `{'localCategories' : 'Hospitals',` line above takes care of that for us. 

**However,** We have not specified a location yet. Unless we want Bing to infer location based on our IP address, we will need to use another url param: `localCircularView`.

In [17]:
newer_params.update({'localCircularView' : '47.6421,-122.13715,5000'})
pprint(newer_params)

{'localCategories': 'Hospitals', 'localCircularView': '47.6421,-122.13715,5000'}


The `'localCircularView'` param draws a circle using a coordinate and a radius. The param takes 3 comma-separated arguments:
 - Latitude of center point
 - Longitude of center point
 - Radius (in meters,) AKA the distance from the center point within which we want to search.

In [18]:
print('lat:{}°\nlong:{}°\nradius:{}m'.format(*newer_params['localCircularView'].split(',')))

lat:47.6421°
long:-122.13715°
radius:5000m


Adding this^ to our existing set of params will define a categorical query for hospitals within 5000 meters of the point 46.6421 by -112.13715. 

For sanity's sake, let's take a quick look at the full list of our url params:

In [19]:
pprint(check_session(q, custom_params=newer_params)[0])

{'count': '50',
 'localCategories': 'Hospitals',
 'localCircularView': '47.6421,-122.13715,5000',
 'mkt': 'en-US',
 'offset': '0',
 'q': ''}


I'm going to say it one more time: **`q` MUST BE AN EMPTY STRING WHEN SEARCHING CATEGORICALLY**. If it isn't, or if you omit it completely, you won't get any results.

Let's try it out:

In [20]:
listings = get_business_listings(q, custom_params=newer_params)
pprint(listings)
print('{} businesses returned with this query'.format(len(listings)))

[('Overlake Medical Center', 'Bellevue'),
 ('Overlake Medical Center & Clinics', 'Bellevue'),
 ('Overlake Radiation Oncology', 'Bellevue'),
 ('Overlake Breast Health Center', 'Bellevue'),
 ('Overlake Medical Center - Senior Health Clinic', 'Bellevue'),
 ('Overlake Hospital Cancer Center Oncology', 'Bellevue'),
 ('Overlake Clinics - Pelvic Health', 'Bellevue'),
 ('Overlake Hospital Cancer Center Mammography and Breast Health Center',
  'Bellevue'),
 ('Pacific Medical Systems Inc', 'Bellevue'),
 ('Kaiser Permanente', 'Bellevue'),
 ("Seattle Children's Bellevue Clinic and Surgery Center", 'Bellevue'),
 ('Overlake Breast Screening Center – Bellevue', 'Bellevue'),
 ('Dermatology Arts', 'Bellevue'),
 ('Surgical Specialists At Overlake', 'Bellevue'),
 ('Bellevue Plastic Surgeons', 'Bellevue'),
 ('Kindred at Home', 'Bellevue'),
 ('Swedish Emergency Room - Redmond', 'Redmond'),
 ('Redmond Urgent Care', 'Redmond'),
 ('Swedish Express Care at Walgreens - Bellevue', 'Bellevue')]
19 businesses retu

Awesome. Let's increment our offset & see if that's all of them.

In [21]:
total_estimated_matches = get_total_estimated_matches(q, custom_params=newer_params)
before = len(listings)

In [22]:
newer_params.update({'offset' : str(len(listings))})
listings += get_business_listings(q, custom_params=newer_params)
listings = dedupe(listings)

In [23]:
after = len(listings)

In [24]:
print('Previously had {} listings, added {} new ones for {} current listings. Bing says {} listings exist'.format(before, after - before, after, total_estimated_matches))

Previously had 19 listings, added 0 new ones for 19 current listings. Bing says 0 listings exist


#### Zooming Out

Finally, let's zoom out and see if we can get more hospitals using a bigger radius value. Previously we used `newer_params.update({'localCircularView' : '47.6421,-122.13715,5000'})` to specify the `localCircularView` param, but rewriting this compound value over and over again is going to be annoying.

Another helper function:

In [25]:
def get_lcv_param(lat, long, rad):
    return ','.join([str(lat), str(long), str(rad)])

In [26]:
new_lcv = get_lcv_param(47.6421, -122.13715, 10000)
print(new_lcv)

47.6421,-122.13715,10000


In [27]:
newer_params.update({'localCircularView' : new_lcv})

Let's reset the count and try again:

In [28]:
newer_params.update({'offset' : '0'})

In [29]:
listings = get_business_listings(q, custom_params=newer_params)

In [31]:
pprint(listings)
print()
print('{} businesses returned with this query'.format(len(listings)))

[('Overlake Medical Center', 'Bellevue'),
 ('Pacific Medical Center', 'Kirkland'),
 ('Overlake Medical Center & Clinics', 'Bellevue'),
 ('Overlake Breast Health Center', 'Bellevue'),
 ('Overlake Radiation Oncology', 'Bellevue'),
 ("Seattle Children's Bellevue Clinic and Surgery Center", 'Bellevue'),
 ('Overlake Hospital Cancer Center Mammography and Breast Health Center',
  'Bellevue'),
 ('EvergreenHealth Family Maternity Center', 'Kirkland'),
 ('Overlake Hospital Cancer Center Oncology', 'Bellevue'),
 ('Overlake Medical Center - Senior Health Clinic', 'Bellevue'),
 ('Overlake Clinics - Pelvic Health', 'Bellevue'),
 ('EvergreenHealth Hospice Care', 'Kirkland'),
 ('EvergreenHealth Heart Care', 'Kirkland'),
 ('Kaiser Permanente', 'Bellevue'),
 ('EvergreenHealth Sleep Services', 'Kirkland'),
 ('Overlake Breast Screening Center – Bellevue', 'Bellevue'),
 ('Pacific Medical Systems Inc', 'Bellevue'),
 ('EvergreenHealth Maternal-Fetal Medicine', 'Kirkland'),
 ('Fairfax Hospital', 'Kirkland'),